<a href="https://colab.research.google.com/github/romellfudi/medium/blob/main/New_Function_calling_with_OpenAI_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://platform.openai.com/docs/guides/function-calling

In [83]:
#@title Installing python dependencies
%%capture
!pip install python-dotenv langchain openai tiktoken faiss-cpu  spotipy -q

In [1]:
#@title To Environment keys { display-mode: "form" }
#@markdown Add your OPEN-API-KEY from [openai](https://openai.com/blog/openai-api)
#@markdown and add your spotify dev credentials from [SpotifyDev](https://developer.spotify.com/dashboard)</br>
#@markdown By default we use a developer-api-key

openai_key = 'sk-'  #@param {type: "string"}
spotipy_client_key = ''  #@param {type: "string"}
spotipy_secret_key = ''  #@param {type: "string"}

with open(".env", "w") as f:
    f.write(f"OPENAI_API_KEY={openai_key}\n")
    f.write(f"SPOTIPY_CLIENT_ID={spotipy_client_key}\n")
    f.write(f"SPOTIPY_CLIENT_SECRET={spotipy_secret_key}\n")

print("Overwriting .env")

from dotenv import load_dotenv
_=load_dotenv()

Overwriting .env


In [84]:
import spotipy
import os
import openai
from spotipy.oauth2 import SpotifyClientCredentials
print("openai version:",openai.__version__)
# Set up your Spotify API credentials
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

openai version: 1.10.0


In [3]:
import json
from pprint import pprint

def list_latest_episode_of_spotify_show(show_name, limit=40):
    """Get the latest episodes of a determine show or episode in a given name"""
    result_fetch = sp.search(show_name, limit=limit, offset=0, type='episode',market='ES')
    final_list = []
    for item in result_fetch['episodes']['items']:
        if show_name.lower() in item['description'].lower():
            final_list.append({'release_date':item['release_date'],'episode_name':item['name'],'url':item['external_urls']['spotify']})
    final_list = sorted(final_list, key=lambda x: x['release_date'], reverse=True)
    return json.dumps(final_list[:limit])

In [4]:
result_fetch = list_latest_episode_of_spotify_show('Globant')
pprint(result_fetch)

('[{"release_date": "2024-01-17", "episode_name": "Inteligencia Artificial: '
 'Esteban Molina - Site Manager Globant - \\u00bfC\\u00f3mo utilizar la '
 'inteligencia artificial en los negocios para aumentar las capacidades '
 'humanas?", "url": '
 '"https://open.spotify.com/episode/21ijkaaME4x9JUpMAnmcsS"}, {"release_date": '
 '"2023-12-26", "episode_name": "Apertura: \\u00bfQu\\u00e9 pas\\u00f3 con '
 'Globant? | #Segurola", "url": '
 '"https://open.spotify.com/episode/6pCoduFRF3twt4za9FQcxk"}, {"release_date": '
 '"2023-12-13", "episode_name": "H2oh! - IA al servicio de la creatividad: el '
 'impulso de las marcas con Pepe Chamorro de GLOBANT Create Studio", "url": '
 '"https://open.spotify.com/episode/2vVH0rB4gNw9dz7UdFxzq3"}, {"release_date": '
 '"2023-11-27", "episode_name": "Vinculando el futuro: empresas y startups en '
 'sinergia I Caso Globant Ventures + Uali", "url": '
 '"https://open.spotify.com/episode/7Isc8QEOhvAwOwi06e81gu"}, {"release_date": '
 '"2023-09-07", "episode_n

In [17]:
tools = [
    {
            "type":"function",
            "function": {
                "name": "list_latest_episode_of_spotify_show",
                "description": "Get the latest episodes  from Spotify shows or episodes in a given name",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "show_name": {
                            "type": "string",
                            "description": "The name of the show or episode, e.g., Apple, Globant, Twitter, Amazon",
                        },
                        "limit": {
                            "type": "integer",
                            "description": "The maximum number of items to fetch",
                        },

                    },
                    "required": ["show_name"],
                },
            }
    }
]

In [18]:
messages = [{'role': 'user', 'content': 'Hello, I need your help to find postcasts in my Spotify'}]
user_message="What are the latest ten episodes of Globant?"
messages.append({"role": "user", "content": user_message})
client = openai.OpenAI()
response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools = tools,
        temperature=0.3,
    )

In [19]:
response_function_calling = response.choices[0].message.tool_calls
response_function_calling

[ChatCompletionMessageToolCall(id='call_LCJoMMEazTWrJbBXYaHrPdpR', function=Function(arguments='{"show_name":"Globant","limit":10}', name='list_latest_episode_of_spotify_show'), type='function')]

In [24]:
function_name = response_function_calling[0].function.name
print(function_name)

list_latest_episode_of_spotify_show


In [26]:
import json
show_name=eval(response_function_calling[0].function.arguments)['show_name']
print(show_name)
limit=eval(response_function_calling[0].function.arguments)['limit']
print(limit)

Globant
10


In [28]:
arguments = json.loads(response_function_calling[0].function.arguments)
arguments

{'show_name': 'Globant', 'limit': 10}

In [29]:
available_functions = {
    'list_latest_episode_of_spotify_show': list_latest_episode_of_spotify_show
}

In [31]:
function_response = available_functions[function_name](
    show_name=arguments.get("show_name"),
    limit=arguments.get("limit"),
)
pprint(function_response, width=150)

('[{"release_date": "2024-01-17", "episode_name": "Inteligencia Artificial: Esteban Molina - Site Manager Globant - \\u00bfC\\u00f3mo utilizar la '
 'inteligencia artificial en los negocios para aumentar las capacidades humanas?", "url": '
 '"https://open.spotify.com/episode/21ijkaaME4x9JUpMAnmcsS"}, {"release_date": "2023-12-26", "episode_name": "Apertura: \\u00bfQu\\u00e9 pas\\u00f3 '
 'con Globant? | #Segurola", "url": "https://open.spotify.com/episode/6pCoduFRF3twt4za9FQcxk"}, {"release_date": "2023-08-18", "episode_name": '
 '"[Noticias del d\\u00eda] A la espera del desembolso del FMI + Licitaci\\u00f3n en pesos + Globant + \\u00edndices en EE.UU. (18/8)", "url": '
 '"https://open.spotify.com/episode/3cjoYVSjR2maPfHZ3XtAod"}, {"release_date": "2023-08-03", "episode_name": "#737 Mauricio Salvatierra: En Globant '
 'buscamos conquistar al mundo desde el lugar de origen de las personas que trabajamos all\\u00ed", "url": '
 '"https://open.spotify.com/episode/5AVOUUK2ecxi5GedSIcww8"}]

In [33]:
result_data = list_latest_episode_of_spotify_show('Google', limit=50)
result_list = json.loads(result_data)
pprint([data['episode_name'] for data in result_list],width=150)

['Google and HuggingFace Partner in Boost to Open Source AI',
 'Google’s Announcements Show 2 Key 2024 AI Trends',
 'Google SGE 2024: Das muss jeder Marketer wissen',
 'Google Glass Half Empty with Leah Rudick and Rod Thill | 18',
 'Ep412 - Nick Lane | The Deep Chemistry of Life and Death',
 'Google landlords | Spy cam | AI copyright',
 'How to be more innovative | Sam Schillace (Microsoft deputy CTO, creator of Google Docs)',
 '#110 Googlet einfach selber',
 'De IA Generativa aos dados: o melhor do Google Cloud Cast Apresenta em 2023',
 'Ó sogra, deixei-se disso e vá pesquisar no Google porquê!',
 'Meta and Google Cut DEI Programs & Can TikTok Save Peloton?',
 'The 14th Annual Google Zeitgeist Review of the Year Show 2023',
 'Ep407 - Cole Nussbaumer Knaflic | Storytelling with Data',
 'Cérebro da Lu: como o Magalu está usando IA generativa no atendimento ao cliente',
 'How to Get to the Top of Google Again and Again and AGAIN',
 'Prof G Markets: Epic Defeats Google, Ohtani’s Dodgers C

In [55]:
%%writefile spotify_utils.py
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def list_latest_episode_of_spotify_show(show_name, limit=20):
    """Get the latest episodes of a determine show or episode in a given name"""
    result_fetch = sp.search(show_name, limit=40, offset=0, type='episode',market='ES')
    final_list = []
    for item in result_fetch['episodes']['items']:
        if show_name.lower() in item['description'].lower():
            final_list.append({'release_date':item['release_date'],'episode_name':item['name'],'url':item['external_urls']['spotify']})
    final_list = sorted(final_list, key=lambda x: x['release_date'], reverse=True)
    print("final_list",final_list)
    return json.dumps(final_list[:limit])

Overwriting spotify_utils.py


In [56]:
import spotify_utils
from spotify_utils import list_latest_episode_of_spotify_show
available_functions = {func_name: func for func_name, func in globals().items() if callable(func) and func_name != 'function_object' and func.__module__ == spotify_utils.__name__}
available_functions

{'list_latest_episode_of_spotify_show': <function spotify_utils.list_latest_episode_of_spotify_show(show_name, limit=20)>}

In [36]:
import spotify_utils
function_object = getattr(spotify_utils, function_name)
function_response = function_object(**arguments)
pprint(function_response, width=150)

('[{"release_date": "2024-01-17", "episode_name": "Inteligencia Artificial: Esteban Molina - Site Manager Globant - \\u00bfC\\u00f3mo utilizar la '
 'inteligencia artificial en los negocios para aumentar las capacidades humanas?", "url": '
 '"https://open.spotify.com/episode/21ijkaaME4x9JUpMAnmcsS"}, {"release_date": "2023-12-26", "episode_name": "Apertura: \\u00bfQu\\u00e9 pas\\u00f3 '
 'con Globant? | #Segurola", "url": "https://open.spotify.com/episode/6pCoduFRF3twt4za9FQcxk"}, {"release_date": "2023-12-13", "episode_name": '
 '"H2oh! - IA al servicio de la creatividad: el impulso de las marcas con Pepe Chamorro de GLOBANT Create Studio", "url": '
 '"https://open.spotify.com/episode/2vVH0rB4gNw9dz7UdFxzq3"}, {"release_date": "2023-11-27", "episode_name": "Vinculando el futuro: empresas y '
 'startups en sinergia I Caso Globant Ventures + Uali", "url": "https://open.spotify.com/episode/7Isc8QEOhvAwOwi06e81gu"}, {"release_date": '
 '"2023-09-07", "episode_name": "Los inicios del UX en

In [78]:
def askToSpotifyBot(user_input, functions=tools):
    """
    Interacts with a Spotify bot to find podcasts and return a list with markdown links.

    Args:
        user_input (str): The user's input/query.
        functions (dict): A dictionary containing available functions for interaction.

    Returns:
        str: Formatted response containing podcast list or function output.
    """

    # Initial system message
    messages = [{'role': 'system', 'content': """
        You are an assistant helping me find podcasts on Spotify and returning the entire list.
        Please reply in markdown format and add the link at the end of each episode using [Listen here](url)""" }]

    # Add user input to messages
    messages.append({"role": "user", "content": user_input})

    # Call the GPT-3 model for initial completion
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools = tools,
        temperature=0.3,
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    messages.append(response_message)
    if tool_calls:
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(**function_args)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response.choices[0].message.content
    else:
        return response_message.content

In [81]:
from IPython.display import Markdown
Markdown(askToSpotifyBot("What are the latest 7 episodes of Twitter? Be concise"))

Here are the latest 7 episodes of Twitter:

1. [The Other Side of Manitham EXPOSED! | Twitter Time Pass Space](https://open.spotify.com/episode/0fazZNw2wCdJZPfzcqeVd0)
2. [Morattu Fun Conversation Between Robinson & Tamil Pulavar Manitham | Twitter Time Pass Space](https://open.spotify.com/episode/4HsbNAs36hGrx82vzgSdAz)
3. [Is Onam A Tamil Festival - Fun Analysis | Twitter Time Pass Space](https://open.spotify.com/episode/3PkX4XN46h6waRaO6HI1qd)
4. [Salaar > Ayalaan & Captain Miller? | Twitter Time Pass Space](https://open.spotify.com/episode/1o0Oyzd1kwOZjKpdZH6ji1)
5. [Vidaamuyarchi CANT Cross The Box Office Collection Of LEO | Twitter Time Pass Space](https://open.spotify.com/episode/5QDIonkpCKmq7kvPMhRbeK)
6. [Ayalaan - The Real Pongal Winner? | Twitter Time Pass Space](https://open.spotify.com/episode/6CUb2AGyLrxJjNRwsFhoc9)
7. [Arun Vijay's Mission Movie Review | Twitter Time Pass Space](https://open.spotify.com/episode/5EbmocyRk3PsX67csM00PH)

In [80]:
Markdown(askToSpotifyBot("What are the latest 8 episodes of Globant? Group by year"))

### 2024
1. **Inteligencia Artificial: Esteban Molina - Site Manager Globant - ¿Cómo utilizar la inteligencia artificial en los negocios para aumentar las capacidades humanas?** - *Jan 17, 2024* - [Listen here](https://open.spotify.com/episode/21ijkaaME4x9JUpMAnmcsS)

### 2023
1. **Apertura: ¿Qué pasó con Globant? | #Segurola** - *Dec 26, 2023* - [Listen here](https://open.spotify.com/episode/6pCoduFRF3twt4za9FQcxk)
2. **H2oh! - IA al servicio de la creatividad: el impulso de las marcas con Pepe Chamorro de GLOBANT Create Studio** - *Dec 13, 2023* - [Listen here](https://open.spotify.com/episode/2vVH0rB4gNw9dz7UdFxzq3)
3. **Vinculando el futuro: empresas y startups en sinergia I Caso Globant Ventures + Uali** - *Nov 27, 2023* - [Listen here](https://open.spotify.com/episode/7Isc8QEOhvAwOwi06e81gu)
4. **Los inicios del UX en Chile con Javier Velasco - Globant Chile** - *Sep 7, 2023* - [Listen here](https://open.spotify.com/episode/5QGuIxYRARBvs5jmMGG4a3)
5. **#Perros2023 - ¿Levantar de vacaciones es más fácil? ¿Tu oficina tiene un toro mecánico? | Apertura 29 de agosto desde Globant** - *Aug 29, 2023* - [Listen here](https://open.spotify.com/episode/7EcmSwjBUeEOyq2tMDO6oO)
6. **[Noticias del día] A la espera del desembolso del FMI + Licitación en pesos + Globant + índices en EE.UU. (18/8)** - *Aug 18, 2023* - [Listen here](https://open.spotify.com/episode/3cjoYVSjR2maPfHZ3XtAod)
7. **PlaybyPlay Podcast: Globant (NYSE: GLOB) and LaLiga Tech work with Microsoft (NasdaqGS: MSFT) and DICK's Sporting Goods (NYSE: DKS) Opens Ni** - *Aug 18, 2023* - [Listen here](https://open.spotify.com/episode/7vPt9ORh7T50Byj9bC2N1k)

In [82]:
Markdown(askToSpotifyBot("What are the latest 15 episodes of Bitcoin? Group by month and change 'Listen here' link to 'Escuchame'"))

### January 2024
- [Accelerate Bitcoin with Dylan LeClair - WBD765](https://open.spotify.com/episode/1e38QQgYDKmct8CJjAB7RF) - Escuchame
- [Trailer: Was erwartet euch bei Bitcoin verstehen?](https://open.spotify.com/episode/3SX2pVoRHKhqsfsmQGJNDk) - Escuchame
- [What’s next for Bitcoin ETFs?](https://open.spotify.com/episode/5xSt2H27sARbAB6fjPf8m3) - Escuchame
- [The AI Election + Bitcoin’s Wall Street Debut + TikTok’s Doodad Era](https://open.spotify.com/episode/5zP2qNLH5bXIQo9IzM9lVf) - Escuchame
- [Bitcoin or Gold? Oaktree’s Howard Marks Sees Little Difference](https://open.spotify.com/episode/6BfV4926tgT3pQ1Wa71rSX) - Escuchame
- [The Orange Pill with Dave Smith - WBD763](https://open.spotify.com/episode/4tYlW7XDfhZ5RuzYyHPPsk) - Escuchame
- [Orange Pilling Wall Street with Hunter Horsley - WBD762](https://open.spotify.com/episode/2bwSRPIf76AWFJznpRD3g8) - Escuchame
- [Bitcoin's Long Journey to the Stock Market](https://open.spotify.com/episode/3AuLbbYFjH4RDMIM3BQ77W) - Escuchame
- [Prof G Markets: Carta and the Secondary Market, Bitcoin ETFs, and Scott’s 2024 Investment Strategy](https://open.spotify.com/episode/1J8wlMsccG3KZtPbdQ5o3v) - Escuchame
- [The Economics of Bitcoin Scaling with Lyn Alden - WBD761](https://open.spotify.com/episode/0bmCBlqJLbzqUhOzideNLu) - Escuchame
- [The Arrival of Bitcoin ETFs with James Seyffart & Alex Thorn - WBD760](https://open.spotify.com/episode/4B3u7oLQHTZq5OMc1LPIhj) - Escuchame
- [Don Lemon Returns, Bitcoin ETF Approval, and BI vs. Axel Springer](https://open.spotify.com/episode/7FdwEMjfMasBCPjy48zS55) - Escuchame
- [A Christmas cracker for some UK retailers, Boeing shares fall after door blowout and Bitcoin ETFs given green light by US regulator](https://open.spotify.com/episode/2g5d4JLTM6fUU06CxkDFwy) - Escuchame
- [What ETFs mean for bitcoin](https://open.spotify.com/episode/5imQkau8b4udv0Tdwrd5to) - Escuchame
- [The Genesis of Bitcoin with Aaron Van Wirdum - WBD759](https://open.spotify.com/episode/7plvH8yNSMFSn1Rq6NWizc) - Escuchame

Let me know if I can help you with anything else!